In [20]:
import logging
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd

# In Jupyter, __file__ is not defined, so use the current working directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
# from frankenstein.graph import FrankensteinGraph
import matcher

# Suppress all logging from FrankensteinGraph and its dependencies
logging.getLogger().setLevel(logging.ERROR)

# Call a pointless matplotlib function to stop it being autoremoved by Jupyter
plt.figure(figsize=(10, 6))

df = pd.read_json('../eval/runs/gpt-4o-mini_answerable-full.jsonl', orient='records', lines=True)
# df = pd.read_json('../eval/runs/gpt-4.1-mini_answerable-full.jsonl', orient='records', lines=True)
# df = pd.read_json('../eval/runs/Llama-3.2-3B-Instruct_answerable-full.jsonl', orient='records', lines=True)


m = matcher.Matcher()

<Figure size 1000x600 with 0 Axes>

In [21]:
df.head()

,id,question_template,question,actions,answer,slot_values,answerable,data_availability,answer_format,messages,pred,correct,error
0,ca004be3-b04a-47dd-9233-3c46fb3d2db1,AverageChange,"For each year between 2010 and 2015, what was ...","[{'name': 'get_country_code_from_name', 'argum...",836871.8,"{'property': 'IS.SHP.GOOD.TU', 'subject': 'VNM...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",836871.8,1.0,0.0
1,f4a9740a-a032-49ff-8e86-35ebd8d91dae,AverageChange,"For each year between 2005 and 2016, what was ...","[{'name': 'get_country_code_from_name', 'argum...",54.507727,"{'property': 'PA.NUS.FCRF', 'subject': 'MWI', ...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",54.50772727272727,1.0,0.0
2,3dd0a72d-a00e-411a-bef6-60a53dad11f6,AverageChange,"For each year between 2009 and 2022, what was ...","[{'name': 'get_country_code_from_name', 'argum...",-1.454708,"{'property': 'FM.LBL.BMNY.ZG', 'subject': 'JOR...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",-1.4547080490058126,1.0,0.0
3,1dd26f5d-f5bd-473b-8079-194da28d097a,AverageChange,"For each year between 2014 and 2018, what was ...","[{'name': 'get_country_code_from_name', 'argum...",-18345.5,"{'property': 'AG.LND.CREL.HA', 'subject': 'GBR...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",-18345.5,1.0,0.0
4,4c7f607e-2025-4033-8c42-81308dd53af4,AverageChange,What was the average yearly change in Afghanis...,"[{'name': 'get_country_code_from_name', 'argum...",-17.9,"{'property': 'AG.YLD.CREL.KG', 'subject': 'AFG...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",-21.48,0.0,20.0


In [22]:
# Count number of nan values
df['correct'].isna().sum()

np.int64(5)

In [23]:
# Value count for non-nan values
df['correct'].dropna().groupby(df['question_template']).value_counts()

question_template           correct
AverageChange               1.0        15
                            0.0         5
AverageProperty             1.0        19
                            0.0         1
AveragePropertyComparison   1.0        17
                            0.0         1
CountryPropertyComparison   1.0        18
                            0.0         1
CountryThresholdCount       0.0        12
                            1.0         8
FactorIncreaseComparison    1.0        14
                            0.0         6
IncreasePropertyComparison  0.0        13
                            1.0         7
PropertyOfSubject           1.0        18
                            0.0         2
PropertyRatioComparison     1.0        18
                            0.0         2
RankChange                  0.0        15
                            1.0         3
RegionComparison            1.0        19
                            0.0         1
RegionComparisonResult      1.0        1

In [24]:
df['correct'].groupby(df['question_template']).value_counts()

question_template           correct
AverageChange               1.0        15
                            0.0         5
AverageProperty             1.0        19
                            0.0         1
AveragePropertyComparison   1.0        17
                            0.0         1
CountryPropertyComparison   1.0        18
                            0.0         1
CountryThresholdCount       0.0        12
                            1.0         8
FactorIncreaseComparison    1.0        14
                            0.0         6
IncreasePropertyComparison  0.0        13
                            1.0         7
PropertyOfSubject           1.0        18
                            0.0         2
PropertyRatioComparison     1.0        18
                            0.0         2
RankChange                  0.0        15
                            1.0         3
RegionComparison            1.0        19
                            0.0         1
RegionComparisonResult      1.0        1